In [1]:
from sympy import symbols, Matrix, rot_axis1, rot_axis2, rot_axis3, latex

def Code(mtx, head):
    ret = head + ':\n'
    for i in range(3):
        text = latex(mtx[i, :3], fold_func_brackets=True, mul_symbol=' * ', mat_delim='')
        res = text.replace('\\begin{matrix}', '').replace('\\end{matrix}', '').replace(' &', ',') \
                  .replace('\\sin {θ_{x}}', 'sx').replace('\\cos {θ_{x}}', 'cx') \
                  .replace('\\sin {θ_{y}}', 'sy').replace('\\cos {θ_{y}}', 'cy') \
                  .replace('\\sin {θ_{z}}', 'sz').replace('\\cos {θ_{z}}', 'cz').replace(', - ', ', -')
        if (res[:2] == '- '):
            res = '-' + res[2:]
        ret += 'row(' + str(i) + ') << ' + res + ';\n'
    return ret

# Variables
tx, ty, tz = symbols('t_x, t_y, t_z')
θx, θy, θz = symbols('θ_x, θ_y, θ_z')
# sympy provides rotation matrices of rotating the axes
# but we need to rotate the objects instead -> nagaive angles
Rx, Ry, Rz = rot_axis1(-θx), rot_axis2(-θy), rot_axis3(-θz)
R = Rx * Ry * Rz
t = Matrix([tx, ty, tz])
p = Matrix([tx, ty, tz, θx, θy, θz])

# Cell p
μpx, μpy, μpz = symbols('μ_{px}, μ_{py}, μ_{pz}')
a, b, c, d, e, f = symbols('a, b, c, d, e, f')
μp = Matrix([μpx, μpy, μpz])
Σp = Matrix([[a, b, c], [b, d, e], [c, e, f]])

# Derivatives of R
dRdθx, dRdθy, dRdθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
ddRdθxθx, ddRdθxθy, ddRdθxθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
ddRdθyθy, ddRdθyθz, ddRdθzθz = Matrix.zeros(3), Matrix.zeros(3), Matrix.zeros(3)
for i in range(3):
    for j in range(3):
        dRdθx[i, j] = R[i, j].diff(θx)
        dRdθy[i, j] = R[i, j].diff(θy)
        dRdθz[i, j] = R[i, j].diff(θz)
        ddRdθxθx[i, j] = R[i, j].diff(θx).diff(θx)
        ddRdθxθy[i, j] = R[i, j].diff(θx).diff(θy)
        ddRdθxθz[i, j] = R[i, j].diff(θx).diff(θz)
        ddRdθyθy[i, j] = R[i, j].diff(θy).diff(θy)
        ddRdθyθz[i, j] = R[i, j].diff(θy).diff(θz)
        ddRdθzθz[i, j] = R[i, j].diff(θz).diff(θz)


# First Derivatives of μpq = R * μp + t  - μq
jtx = Matrix([1, 0, 0])  # dμpqdtx
jty = Matrix([0, 1, 0])  # dμpqdty
jtz = Matrix([0, 0, 1])  # dμpqdtz
jθx = dRdθx * μp         # dμpqdθx
jθy = dRdθy * μp         # dμpqdθy
jθz = dRdθz * μp         # dμpqdθz

# Second Derivatives of μpq
# Derivative that contains dtx, dty, or dtz is zero
Hθxθx = ddRdθxθx * μp    # ddμpqdθxdθx
Hθxθy = ddRdθxθy * μp    # ddμpqdθxdθy
Hθxθz = ddRdθxθz * μp    # ddμpqdθxdθz
Hθyθy = ddRdθyθy * μp    # ddμpqdθydθy
Hθyθz = ddRdθyθz * μp    # ddμpqdθydθz
Hθzθz = ddRdθzθz * μp    # ddμpqdθzdθz

# First Derivatives of Σpq = R * Σp * R.T + Σq
# Derivative that contains dtx, dty, or dtz is zero
Zθx = (dRdθx * Σp * R.T) + (dRdθx * Σp * R.T).T  # dΣpqdθx
Zθy = (dRdθy * Σp * R.T) + (dRdθy * Σp * R.T).T  # dΣpqdθy
Zθz = (dRdθz * Σp * R.T) + (dRdθz * Σp * R.T).T  # dΣpqdθz

# Second Derivatives of Σpq
# Derivative that contains dtx, dty, or dtz is zero
Zθxθx = (ddRdθxθx * Σp * R.T + dRdθx * Σp * dRdθx.T) + (ddRdθxθx * Σp * R.T + dRdθx * Σp * dRdθx.T).T  # ddΣpqdθxdθx
Zθxθy = (ddRdθxθy * Σp * R.T + dRdθx * Σp * dRdθy.T) + (ddRdθxθy * Σp * R.T + dRdθx * Σp * dRdθy.T).T  # ddΣpqdθxdθy
Zθxθz = (ddRdθxθz * Σp * R.T + dRdθx * Σp * dRdθz.T) + (ddRdθxθz * Σp * R.T + dRdθx * Σp * dRdθz.T).T  # ddΣpqdθxdθz
Zθyθy = (ddRdθyθy * Σp * R.T + dRdθy * Σp * dRdθy.T) + (ddRdθyθy * Σp * R.T + dRdθy * Σp * dRdθy.T).T  # ddΣpqdθydθy
Zθyθz = (ddRdθyθz * Σp * R.T + dRdθy * Σp * dRdθz.T) + (ddRdθyθz * Σp * R.T + dRdθy * Σp * dRdθz.T).T  # ddΣpqdθydθz
Zθzθz = (ddRdθzθz * Σp * R.T + dRdθz * Σp * dRdθz.T) + (ddRdθzθz * Σp * R.T + dRdθz * Σp * dRdθz.T).T  # ddΣpqdθzdθz

In [74]:
print(Code(R, 'R'))
print(Code(dRdθx, 'dRdθx'))
print(Code(dRdθy, 'dRdθy'))
print(Code(dRdθz, 'dRdθz'))
print(Code(ddRdθxθx, 'ddRdθxθx'))
print(Code(ddRdθxθy, 'ddRdθxθy'))
print(Code(ddRdθxθz, 'ddRdθxθz'))
print(Code(ddRdθyθy, 'ddRdθyθy'))
print(Code(ddRdθyθz, 'ddRdθyθz'))
print(Code(ddRdθzθz, 'ddRdθzθz'))

R:
row(0) << cy * cz, -sz * cy, sy;
row(1) << sx * sy * cz + sz * cx, -sx * sy * sz + cx * cz, -sx * cy;
row(2) << sx * sz - sy * cx * cz, sx * cz + sy * sz * cx, cx * cy;

dRdθx:
row(0) << 0, 0, 0;
row(1) << -sx * sz + sy * cx * cz, -sx * cz - sy * sz * cx, -cx * cy;
row(2) << sx * sy * cz + sz * cx, -sx * sy * sz + cx * cz, -sx * cy;

dRdθy:
row(0) << -sy * cz, sy * sz, cy;
row(1) << sx * cy * cz, -sx * sz * cy, sx * sy;
row(2) << -cx * cy * cz, sz * cx * cy, -sy * cx;

dRdθz:
row(0) << -sz * cy, -cy * cz, 0;
row(1) << -sx * sy * sz + cx * cz, -sx * sy * cz - sz * cx, 0;
row(2) << sx * cz + sy * sz * cx, -sx * sz + sy * cx * cz, 0;

ddRdθxθx:
row(0) << 0, 0, 0;
row(1) << -sx * sy * cz - sz * cx, sx * sy * sz - cx * cz, sx * cy;
row(2) << -sx * sz + sy * cx * cz, -sx * cz - sy * sz * cx, -cx * cy;

ddRdθxθy:
row(0) << 0, 0, 0;
row(1) << cx * cy * cz, -sz * cx * cy, sy * cx;
row(2) << sx * cy * cz, -sx * sz * cy, sx * sy;

ddRdθxθz:
row(0) << 0, 0, 0;
row(1) << -sx * cz - sy * sz * cx,

In [3]:
def Book(mtx):
    ret = ''
    for i in range(3):
        text = latex(mtx[i, :3], fold_func_brackets=True, mat_delim='')
        res = text.replace('\\begin{matrix}', '').replace('\\end{matrix}', '') \
                  .replace('\\sin {θ_{x}}', 's_x').replace('\\cos {θ_{x}}', 'c_x') \
                  .replace('\\sin {θ_{y}}', 's_y').replace('\\cos {θ_{y}}', 'c_y') \
                  .replace('\\sin {θ_{z}}', 's_z').replace('\\cos {θ_{z}}', 'c_z').replace(', - ', ', -')
        if (res[:2] == '- '):
            res = '-' + res[2:]
        ret += res + '\\\\\n'
    return ret

In [4]:
print(Book(R))
print(Book(dRdθx))
print(Book(dRdθy))
print(Book(dRdθz))
print(Book(ddRdθxθx))
print(Book(ddRdθxθy))
print(Book(ddRdθxθz))
print(Book(ddRdθyθy))
print(Book(ddRdθyθz))
print(Book(ddRdθzθz))

c_y c_z & - s_z c_y & s_y\\
s_x s_y c_z + s_z c_x & - s_x s_y s_z + c_x c_z & - s_x c_y\\
s_x s_z - s_y c_x c_z & s_x c_z + s_y s_z c_x & c_x c_y\\

0 & 0 & 0\\
-s_x s_z + s_y c_x c_z & - s_x c_z - s_y s_z c_x & - c_x c_y\\
s_x s_y c_z + s_z c_x & - s_x s_y s_z + c_x c_z & - s_x c_y\\

-s_y c_z & s_y s_z & c_y\\
s_x c_y c_z & - s_x s_z c_y & s_x s_y\\
-c_x c_y c_z & s_z c_x c_y & - s_y c_x\\

-s_z c_y & - c_y c_z & 0\\
-s_x s_y s_z + c_x c_z & - s_x s_y c_z - s_z c_x & 0\\
s_x c_z + s_y s_z c_x & - s_x s_z + s_y c_x c_z & 0\\

0 & 0 & 0\\
-s_x s_y c_z - s_z c_x & s_x s_y s_z - c_x c_z & s_x c_y\\
-s_x s_z + s_y c_x c_z & - s_x c_z - s_y s_z c_x & - c_x c_y\\

0 & 0 & 0\\
c_x c_y c_z & - s_z c_x c_y & s_y c_x\\
s_x c_y c_z & - s_x s_z c_y & s_x s_y\\

0 & 0 & 0\\
-s_x c_z - s_y s_z c_x & s_x s_z - s_y c_x c_z & 0\\
-s_x s_y s_z + c_x c_z & - s_x s_y c_z - s_z c_x & 0\\

-c_y c_z & s_z c_y & - s_y\\
-s_x s_y c_z & s_x s_y s_z & s_x c_y\\
s_y c_x c_z & - s_y s_z c_x & - c_x c_y\\

s_y s_z